In [105]:
import cv2 as cv
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import os

In [106]:
# Set the path to the data folder
data_path = '../data/'

classes = os.listdir(data_path)
data = []
labels = []

for type in classes:
    all_apples = os.listdir(data_path + type)
    for apple in all_apples:
        img = cv.imread(data_path + type + '/' + apple)
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img = cv.resize(img, (512, 512))
        data.append(img)
        labels.append(type)


In [107]:
# Split data and labels into a training set and a test set
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

## Bag of visual words (BoVW) + SVM

In [108]:
# Step 1: Feature extraction
sift = cv.xfeatures2d.SIFT_create()
descriptors = []
for img in train_data:
    kp, des = sift.detectAndCompute(img, None)
    descriptors.extend(des)


In [109]:
# Step 2: Codebook generation
kmeans = KMeans(n_clusters=350)
kmeans.fit(descriptors)
codebook = kmeans.cluster_centers_


e:\conda\envs\uni\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [110]:
# Step 3: Image representation
def get_histogram(targer_images):
    features = []
    for img in targer_images:
        kp, des = sift.detectAndCompute(img, None)
        histogram = np.zeros(len(codebook))
        for d in des:
            idx = kmeans.predict([d])
            histogram[idx] += 1
        features.append(histogram)
    return features

train_features = get_histogram(train_data)
test_features = get_histogram(test_data)


In [111]:
# Normalize the histograms
scaler = StandardScaler().fit(train_features)
train_features = scaler.transform(train_features)
test_features = scaler.transform(test_features)


In [112]:
# Step 4: Classification with hyperparameter tuning
svm = SVC()

# Define the parameter grid
param_grid = {
    'C': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(train_features, train_labels)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)


Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ....................C=0.1, gamma=scale

In [113]:
# Fit the SVM classifier with the best parameters
svm = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'])
svm.fit(train_features, train_labels)


SVC(C=1.5, kernel='sigmoid')

In [115]:
# Predict the labels for the test data
predicted_labels = svm.predict(test_features)

print("\nAccuracy Score:")
print((accuracy_score(test_labels, predicted_labels))*100, "%")


Accuracy Score:
87.5 %
